# Find Relevant Sentences

## Set up working enviroment

In [3]:
from typing import *
import sqlite3
from sqlite3 import Cursor, Connection
from os import listdir
from os.path import isfile, join

In [4]:
# Initialise DB connection
_data_base: Connection = sqlite3.connect('../models/Model/DB.db')
_cursor: Cursor = _data_base.cursor()

## Data Preprocessing

In [7]:
def build_dataset_test(table: str):
    X = []
    Y = []
    sentence = ''
    relevance = 0
    query = "SELECT Word,TokenSet FROM " + table + " Order BY FileName, Line"
    table_contents = list(_cursor.execute(query))
    for table_content in table_contents:
        word = table_content[0]
        token_set = table_content[1]
        if len(word) == 0:
            if sentence != '':
                X.append(sentence)
                Y.append(relevance)
                sentence = ''
                relevance = 0
        else:
            if token_set == "" or token_set != 'O':
                relevance = 1
            if word in [",", "-", "."]:
                sentence = (sentence + word).strip()
            else:
                sentence = (sentence + ' ' + word).strip()
    return X, Y

In [8]:
train_X, train_Y = build_dataset_test(table="Train_Token")
dev_X, dev_Y = build_dataset_test(table="Dev_Token")
test_X, test_Y = build_dataset_test(table="Test_Token")

ANALYSIS-ON-APT-TO-BE-ATTACK-THAT-FOCUSING-ON-CHINAS-GOVERNMENT-AGENCY-


KeyboardInterrupt: 

In [0]:
print(f'Train size: {len(train_X)}')
print(f'Dev size: {len(dev_X)}')
print(f'Test size: {len(test_X)}')

Train size: 9424
Dev size: 1213
Test size: 618


In [0]:
print(f'[Train] relevant sentences: {train_Y.count(1)}')
print(f'[Train] irrelevant sentences: {train_Y.count(0)}')
print(f'[Train] % relevant: {train_Y.count(1) / len(train_Y) * 100}')
print()

print(f'[Dev] relevant sentences: {dev_Y.count(1)}')
print(f'[Dev] irrelevant sentences: {dev_Y.count(0)}')
print(f'[Dev] % relevant: {dev_Y.count(1) / len(dev_Y) * 100}')
print()

print(f'[Test] relevant sentences: {test_Y.count(1)}')
print(f'[Test] irrelevant sentences: {test_Y.count(0)}')
print(f'[Test] % relevant: {test_Y.count(1) / len(test_Y) * 100}')

[Train] relevant sentences: 2204
[Train] irrelevant sentences: 7220
[Train] % relevant: 23.387096774193548

[Dev] relevant sentences: 79
[Dev] irrelevant sentences: 1134
[Dev] % relevant: 6.512778235779059

[Test] relevant sentences: 90
[Test] irrelevant sentences: 528
[Test] % relevant: 14.563106796116504


In [0]:
import pandas as pd

train = {'text':train_X, 'labels':train_Y}
dev = {'text':dev_X, 'labels':dev_Y}
test = {'text':test_X, 'labels':test_Y}

train_df = pd.DataFrame(train)
dev_df = pd.DataFrame(dev)
test_df = pd.DataFrame(test)

##Relevance Classifier

In [0]:
!pip install simpletransformers

In [0]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [0]:
!sh setup.sh

In [0]:
from simpletransformers.classification import ClassificationModel
import logging
import sklearn


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Optional model configuration
model_args = {
    "num_train_epochs": 5,
    "overwrite_output_dir": True #OJO
}

# Create a ClassificationModel
model = ClassificationModel(
    "roberta", "roberta-base", args=model_args, 
)

In [0]:
# Train the model
model.train_model(train_df)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.715874

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.740841

...

Running loss: 0.399197


Running loss: 0.580149Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0
Running loss: 0.443776


Running loss: 0.358143


Running loss: 0.132167



INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


In [0]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.45395703544341165, 'tp': 80, 'tn': 385, 'fp': 143, 'fn': 10, 'acc': 0.7524271844660194, 'eval_loss': 0.6238705667738731}


In [0]:
print(result)
#print(model_outputs)

{'mcc': 0.45395703544341165, 'tp': 80, 'tn': 385, 'fp': 143, 'fn': 10, 'acc': 0.7524271844660194, 'eval_loss': 0.6238705667738731}


In [0]:
# Make predictions with the model
predictions, raw_outputs = model.predict(["In addition to banking and payment systems, hackers got access to e-mail servers to control all internal communications."]) #Relevant
#predictions, raw_outputs = model.predict(["Later the attackers completely abandoned from usage of RDPdoor and Team Viewer."])
print(predictions)
print(raw_outputs)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.



[1]
[[-0.12237549  0.21643066]]
